In [ ]:
# Data manipulation libraries
import numpy as np
import pandas as pd

In [ ]:
#Data visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sn
import nltk
import sqlalchemy as sa
from sqlalchemy import create_engine

In [ ]:
#NLP libaries
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import wordcloud
from wordcloud import WordCloud, STOPWORDS

In [ ]:
nltk.download('wordnet')

In [ ]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Sicelo'

In [ ]:
#Create the connection

connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database)

In [ ]:
#query database

sql_query = "Select * from Bank_Statement"

In [ ]:
#creating a dataframe

df_bank = pd.read_sql_query(sql_query, connection)

In [ ]:
df_bank.head()

In [ ]:
y=df_bank['Category'].str.lower()
x=df_bank['Description']

In [ ]:
y_unique = y.value_counts()
y_unique = pd.DataFrame(y_unique)
y_unique.head()

In [ ]:
y_unique = y_unique.reset_index(drop=False)

In [ ]:
y_unique.drop(['Category'], axis=1, inplace=True)  
y_unique['New_ID'] = y_unique.index
y_unique.head()

In [ ]:
y =  pd.DataFrame(y)
mapping = dict(y_unique[['index', 'New_ID']].values)
y['Item_ID'] = y.Category.map(mapping)
y.drop(['Category'], axis=1, inplace=True)

In [ ]:
features = df_bank.Description.values
features2 = df_bank.Description.values

In [ ]:
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

# create function to clean text
def clean_text(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    text = [lemmatizer.lemmatize(word) for word in text.split(' ')]
    text = [word for word in text if not word in stop_words]
    text = ' '.join(text)
    return text

X = df_bank['Description'].apply(lambda x: clean_text(x))

In [ ]:
y=df_bank.Category.apply(lambda x: x.lower())

In [ ]:
y_unique = y.value_counts()
y_unique = pd.DataFrame(y_unique)
y_unique = y_unique.reset_index(drop=False)
y_unique.drop(['Category'], axis=1, inplace=True)  
y_unique['New_ID'] = y_unique.index

In [ ]:
y =  pd.DataFrame(y)

In [ ]:
mapping = dict(y_unique[['index', 'New_ID']].values)
y['Item_ID'] = y.Category.map(mapping)
y.drop(['Category'], axis=1, inplace=True)

In [ ]:
features = df_bank.Description.values
features2 = df_bank.Description.values

In [ ]:
nltk.download('wordnet')

In [ ]:
nltk.download('stopwords')

In [ ]:
import re

In [ ]:
lem = WordNetLemmatizer()
# ps = PorterStemmer()
corpus =[]
# cleaning text
for i in range(0,len(features)):

    item = re.sub('[^a-zA-Z]', ' ',str(features)) 
    item = item.lower()
    item = item.split()
    stop_wrds = set(stopwords.words('english'))
    item = [lem.lemmatize(word, "v") for word in item if not word in stop_wrds]
    item = ' '.join(item)
    corpus.append(item)

new_features = corpus

In [ ]:
obj = CountVectorizer()
final_features = obj.fit_transform(new_features)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_features, y, test_size=0.2, random_state=42)

In [ ]:
logmodel=LogisticRegression()
logmodel.fit(X_train, y_train)

logtree=DecisionTreeClassifier()
logtree.fit(X_train,y_train)

logran=RandomForestClassifier()
logran.fit(X_train,y_train)

gau = GaussianNB()
gau.fit(X_train.toarray(),y_train)

svc = LinearSVC(multi_class='crammer_singer', dual=False, fit_intercept=False)
logsvc = svc.fit(X_train, y_train)


In [ ]:
gau_pred=gau.predict(final_features.toarray())


In [ ]:
log_pred=logmodel.predict(final_features)


In [ ]:
tree_pred=logtree.predict(final_features)


In [ ]:
ran_pred=logran.predict(final_features)


In [ ]:
svc_pred=logsvc.predict(final_features)

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
y_test.shape

In [ ]:
print(classification_report(y_test, y_pred))